In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import homogeneity_score

from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.random_projection import GaussianRandomProjection

import seaborn as sns
import matplotlib.pyplot as plt
from random import sample

random_state = 23523
scoring = 'roc_auc_ovo'

C:\Users\rache\miniconda3\envs\cs-7641-env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# --- load

X_bus = pd.read_pickle('data/business-classification/X.pkl')
Y_bus = pd.read_pickle('data/business-classification/Y.pkl')

X_wine = pd.read_pickle('data/wine/X.pkl')
Y_wine = pd.read_pickle('data/wine/Y.pkl')

# --- split

X_bus_train, X_bus_test, Y_bus_train, Y_bus_test = train_test_split(
    X_bus, Y_bus,
    test_size=0.3,
    random_state=random_state
)

X_wine_train, X_wine_test, Y_wine_train, Y_wine_test = train_test_split(
    X_wine, Y_wine,
    test_size=0.3,
    random_state=random_state
)

training_datasets = {
    'BusClass': (X_bus_train, Y_bus_train),
    'Wine': (X_wine_train, Y_wine_train)
}

holdout_datasets = {
    'BusClass': (X_bus_test, Y_bus_test),
    'Wine': (X_wine_test, Y_wine_test)
}

In [3]:
training_datasets['BusClass'][0]

,ability,able,accept,access,accessibility,accessories,account,achieve,act,action,...,worked,working,works,world,worldwide,www,year,years,york,youtube
60710,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,0
31034,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
45323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25823,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4256,0,0,0,0,0,0,1,0,0,1,...,0,1,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66968,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
622,0,1,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,0,1,0,0
62337,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,1,0,0
6112,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [4]:
training_datasets['Wine'][0]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,red
1514,6.9,0.84,0.21,4.10,0.074,16.0,65.0,0.99842,3.53,0.72,9.233333,1
4229,6.3,0.26,0.25,5.20,0.046,11.0,133.0,0.99202,2.97,0.68,11.000000,0
1815,6.8,0.30,0.35,2.80,0.038,10.0,164.0,0.99120,3.09,0.53,12.000000,0
3713,6.6,0.28,0.23,10.40,0.049,45.0,190.0,0.99754,3.12,0.51,8.800000,0
2328,6.9,0.35,0.55,11.95,0.038,22.0,111.0,0.99687,3.11,0.29,9.700000,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4260,6.2,0.36,0.22,5.25,0.038,44.0,145.0,0.99184,3.22,0.40,11.200000,0
622,10.0,0.58,0.22,1.90,0.080,9.0,32.0,0.99740,3.13,0.55,9.500000,1
3394,6.5,0.25,0.45,7.80,0.048,52.0,188.0,0.99576,3.20,0.53,9.100000,0
4513,6.8,0.40,0.29,2.80,0.044,27.0,97.0,0.99040,3.12,0.42,11.200000,0


# Step 1.

## K-Means

In [8]:
# KMeans
scores = {
    'BusClass': pd.DataFrame(columns=['Silhouette', 'CH', 'Homogeneity']),
    'Wine':     pd.DataFrame(columns=['Silhouette', 'CH', 'Homogeneity'])
}
for dataset_name in training_datasets:
    for k in [2, 3, 4, 5, 6]:
        X, y = training_datasets[dataset_name]
        clustering_model = KMeans(
            n_clusters=k,
            random_state=random_state
        )
        clustering_model.fit(X)
        labels = clustering_model.labels_
        sil_score = silhouette_score(X, labels)
        ch_score = calinski_harabasz_score(X.values, labels)
        homog = homogeneity_score(y, labels)
        print(f'Score for k={k} on {dataset_name}: SS={sil_score:.2f} CH={ch_score:.2f}')
        scores[dataset_name].loc[k, 'Silhouette'] = sil_score
        scores[dataset_name].loc[k, 'CH'] = ch_score
        scores[dataset_name].loc[k, 'Homogeneity'] = homog
        

Score for k=2 on BusClass: SS=0.12 CH=2037.27
Score for k=3 on BusClass: SS=0.03 CH=1355.42
Score for k=4 on BusClass: SS=0.03 CH=1028.80
Score for k=6 on BusClass: SS=-0.03 CH=728.30


KeyboardInterrupt: 

In [ ]:
scores['BusClass']

In [ ]:
scores['Wine']

We'll select $k = 2$ for business classification and $k = 4$ for the wine model.

### Validate Clusters

### Graphical Views

In [ ]:
# review some pairwise plots
for dataset_name, k in [ ('BusClass', 2), ('Wine', 4) ]:
    print(f'HERE with {dataset_name}, {k}')
    tmp = training_datasets[dataset_name][0].copy()
    model = KMeans(
        n_clusters=k,
        random_state=random_state
    )
    model.fit(tmp)
    features_of_interest = sample(tmp.columns.to_list(), 3)
    tmp = tmp[features_of_interest]
    tmp.loc[:, 'clustered_labels'] = model.labels_
    for col in tmp:
        tmp.loc[:, col] = np.array(tmp[col])
    tmp.reset_index(drop=True, inplace=True)
    sns.pairplot(tmp, hue="clustered_labels", vars=features_of_interest).set(title=f'{dataset_name} K-Means Pair-plot)
    plt.show()

### For business classification, what industries are generally represented in one cluster?

In [35]:
X = training_datasets['BusClass'][0].copy()
model = KMeans(
    n_clusters=2,
    random_state=random_state
)
model.fit(X)

df = pd.DataFrame({
    'Cluster': model.labels_,
    'Industry Group': training_datasets['BusClass'][1].copy()
})

true_label_proportions = df.groupby(['Cluster', 'Industry Group']).size().unstack(level=0).fillna(0)
true_label_proportions = true_label_proportions.div(true_label_proportions.sum(axis=0), axis=1)
true_label_proportions.round(2).assign(diff = lambda x: x[1] - x[0]).sort_values('diff')

Cluster,0,1,diff
Industry Group,,,
Financials,0.14,0.07,-0.07
Information Technology,0.13,0.06,-0.07
Professional Services,0.12,0.09,-0.03
Healthcare,0.10,0.09,-0.01
Commercial Services & Supplies,0.09,0.09,0.00
Corporate Services,0.09,0.09,0.00
"Media, Marketing & Sales",0.09,0.09,0.00
Energy & Utilities,0.07,0.08,0.01
Consumer Discretionary,0.03,0.04,0.01


Very cool! Although it's hard to see valuable groupings at the individual industry group level, it totally looks like the model was able to separate by white collar work vs. blue collar work.

### For wine quality classification, what qualities are generally represented in one cluster?

In [32]:
X = training_datasets['Wine'][0].copy()
model = KMeans(
    n_clusters=4,
    random_state=random_state
)
model.fit(X)

df = pd.DataFrame({
    'Cluster': model.labels_,
    'Wine Quality': training_datasets['Wine'][1].copy()
})

true_label_proportions = df.groupby(['Cluster', 'Wine Quality']).size().unstack(level=0).fillna(0)
true_label_proportions = true_label_proportions.div(true_label_proportions.sum(axis=0), axis=1).sort_values(0)
true_label_proportions.loc[['Low', 'Medium', 'High']].round(2)

Cluster,0,1,2,3
Wine Quality,,,,
Low,0.33,0.28,0.42,0.50
Medium,0.43,0.46,0.42,0.42
High,0.24,0.25,0.15,0.08


Although it struggled with the medium quality wine, the clusters are able to help separate the low and high quality items

## E-M

In [ ]:
em_scores = {
    'BusClass': pd.DataFrame(columns=['AIC', 'BIC']),
    'Wine':     pd.DataFrame(columns=['AIC', 'BIC'])
}
for dataset_name in training_datasets:
    for k in [2, 3, 4, 6, 12]:
        X, _ = training_datasets[dataset_name]
        clustering_model = GaussianMixture(
            n_components=k,
            random_state=random_state,
            init_params='random_from_data'
        )
        clustering_model.fit(X)
        bic = clustering_model.bic(X)
        aic = clustering_model.aic(X)
        print(f'Score for k={k} on {dataset_name}: AIC={aic:,.2f} BIC={bic:,.2f}')
        em_scores[dataset_name].loc[k, 'AIC'] = aic
        em_scores[dataset_name].loc[k, 'BIC'] = bic
        

### Validate Clusters

# Step 2.

In [ ]:
scores = {
    'BusClass': pd.DataFrame(index=['k'], columns=['Silhouette', 'CH', 'Homogeneity']),
    'Wine':     pd.DataFrame(index=['k'], columns=['Silhouette', 'CH', 'Homogeneity'])
}
for dataset_name in training_datasets:
    for dim_reducer in [PCA, FastICA, GaussianRandomProjection]:
        for k in [2, 3, 4, 6, 12]:
            X, y = training_datasets[dataset_name]
            clustering_model = dim_reducer(
                n_clusters=k,
                random_state=random_state
            )
            clustering_model.fit(X)
            labels = clustering_model.labels_
            sil_score = silhouette_score(X, labels)
            ch_score = calinski_harabasz_score(X, labels)
            homog = homogeneity_score(y, labels)